In [2]:
import pandas as pd
import re
import numpy as np

from functools import reduce
import nltk

In [3]:
import stop_words

In [4]:
import gensim
from gensim.models.doc2vec import LabeledSentence

In [5]:
import math

In [6]:
import random

## for cleanup

In [7]:
ps = nltk.stem.snowball.SnowballStemmer('english')

In [8]:
sw = stop_words.get_stop_words('english')

In [9]:
regex = re.compile('[^a-zA-Z]')
#First parameter is the replacement, second parameter is your input string
regex.sub('', 'ab3d*E')

'abdE'

In [10]:
def clean_query(tq):
    result = " "
    t = regex.sub(' ',tq)
    for w in t.split():
        if w not in sw:
            result += ps.stem(w.replace(r'[^a-zA-Z]', '')) + " "
    return result

In [11]:
def prepare_query(tq):
    result = " "
    t = regex.sub(' ',tq)
    for w in t.split():
        if w not in sw:
            result += ps.stem(w.replace(r'[^a-zA-Z]', '')) + " "
    return result.split()

## prepare data

In [12]:
class Doc:
    def __init__(self, title, author, text_lines):
        self.title = title
        self.author=author
        self.text = flat(list(map(lambda l: regex.sub(' ',l).split(), text_lines)))

In [13]:
def flat(list_of_list):
    result = []
    for li in list_of_list:
        for el in li:
            result.append(el)
            
    result = list(filter(lambda l:len(line) > 1, result))
    
    return result

In [14]:
def parse_article_lines(lines):
    text_index = 0
    
    l_s_i = 0
    l_e_i = 0 # label_{start-end}_index
    
    for i,l in enumerate(lines):
        if '.T' in l:
            l_s_i = i+1
        
        if '.A' in l:
            l_e_i = i
        
        if '.W' in l:
            text_index = i+1
    
    author = lines[l_e_i+1]
    
    return Doc(lines[l_s_i:l_e_i], author, lines[text_index:len(lines)])

In [15]:
article_start = 0
article_lines = []

docs = {}

lines = []
with open('Documents.csv') as file:
    lines = file.readlines()

for i,line in enumerate(lines):
    if 'Id' in line and len(line) < 12 and article_start != i:
        article_lines = lines[article_start:i]
        article_start = i
        
        q_id = line.split()[1]
        docs[q_id] = parse_article_lines(article_lines)
    

In [16]:
rel_train = pd.read_csv('relevance_train.csv', sep=',', lineterminator='\n')
rel_test = pd.read_csv('relevance_test.csv')
queries = pd.read_csv('queries.csv', sep='|',lineterminator='\n')

In [17]:
def doc2labeled_sentence(doc):
    return LabeledSentence(words=doc.text, tags=doc.title)

In [18]:
ls = list(map(doc2labeled_sentence, docs.values()))

In [19]:
t = list(map(lambda d: d.text, docs.values()))

docs_as_arr = []
for d in t:
    if len(d) > 0:
        docs_as_arr.append(reduce(lambda w1,w2: w1 + " " + w2 ,d))

#text_as_str = reduce(lambda w1,w2 : w1 + " " + w2, t)

## now prepare model

In [19]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)  

FileNotFoundError: [Errno 2] No such file or directory: 'GoogleNews-vectors-negative300.bin'

In [89]:
SIZE = 10

In [72]:
model = gensim.models.Word2Vec(min_count=1,workers=4, hs=1, negative=0)

In [73]:
model.build_vocab(flat(t))

In [80]:
model = gensim.models.Doc2Vec(size=SIZE, min_count=1, workers=4, hs=1, negative=0)

In [81]:
model.build_vocab(ls)

In [29]:
kw = model.wv

NameError: name 'model' is not defined

In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [91]:
tv_docs = TfidfVectorizer(max_features=SIZE, ngram_range=(1,3), stop_words='english')

In [92]:
tv_docs.fit(flat(t))

Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x7f8d23dcfc18>>
Traceback (most recent call last):
  File "/home/lyan/anaconda3/lib/python3.6/site-packages/xgboost/core.py", line 324, in __del__
    _check_call(_LIB.XGDMatrixFree(self.handle))
AttributeError: 'DMatrix' object has no attribute 'handle'


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [93]:
tv_queries = TfidfVectorizer(max_features=SIZE, ngram_range=(1,3), stop_words='english')

In [94]:
tv_queries.fit(queries.iloc[:,1].values)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

## fix that, i suppose should be some library method or somehow fix vocaulary

In [27]:
def get_sen_vector(sen):
    v = np.zeros(SIZE)
    for w in sen:
        try:
            v = np.add(v, kw[w])
        except ValueError as e:
            pass
        except KeyError as e:
            pass
        
    return v / SIZE

## prepare now pairs doc - request

In [28]:
vectors = []
labels = []

for row in rel_train.iterrows():
    qid = row[1][0]
    did = row[1][1]
    rel = row[1][2]
    
    if str(did) in docs and qid in queries['QueryId']:
            labels.append(rel)
            q_v = get_sen_vector(queries[queries['QueryId']==qid])
            d_v = get_sen_vector(docs[str(did)].text)
            vectors.append(np.add(q_v,d_v) / 2.0)
        
vectors = np.array(vectors)
labels = np.array(labels).reshape(len(labels),1)

NameError: name 'kw' is not defined

## same for tfidf from sklearn

In [95]:
vectors = []
labels = []

for row in rel_train.iterrows():
    qid = row[1][0]
    did = row[1][1]
    rel = row[1][2]
    
    if str(did) in docs and qid in queries['QueryId']:
            labels.append(rel)
            q_v = np.mean(tv_queries.transform(queries[queries['QueryId']==qid]), axis=0)
            d_v = np.mean(tv_docs.transform(docs[str(did)].text), axis=0)
            r = np.hstack((q_v, d_v)).reshape(1, 2 * SIZE)
            vectors.append(r)
        
vectors = np.array(vectors).reshape(len(vectors), 2 * SIZE)
labels = np.array(labels).reshape(len(labels),1)

## metrics

In [31]:
def dcg(rels,p):
    s = 0.0
    for i in range(0,p):
        rel = rels[i]
        s += (pow(rel,2)-1) / (math.log(i+1,2)+1)
        
    return s

In [32]:
def ndcg(ideal, rel, p):
    return dcg(rel,p) / dcg(ideal,len(ideal))

## prepare classification model

In [33]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

/home/lyan/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [96]:
trainX, testX, trainY, testY = train_test_split(vectors, labels)

In [97]:
dtrain = xgb.DMatrix(trainX, trainY)
dtest = xgb.DMatrix(testX, testY)

In [102]:

param = {'silent':1, 'objective':'rank:pairwise' }
param['nthread'] = 3
param['eval_metric'] = 'ndcg'
param['eta'] = 0.075
param['max_depth'] = 5
param['silent'] = 1
param['n_estimators'] = 500
param['early_stopping_rounds'] = 30

watchlist = [ (dtrain,'train'), (dtest, 'test') ]

In [103]:
bst = xgb.train(param, dtrain, 1000, watchlist,verbose_eval=100)

[0]	train-:0.538539	test-:0.487108
[100]	train-:0.509153	test-:0.463131
[200]	train-:0.505934	test-:0.460353
[300]	train-:0.504962	test-:0.458292
[400]	train-:0.503897	test-:0.457401
[500]	train-:0.504294	test-:0.458746
[600]	train-:0.503843	test-:0.456646
[700]	train-:0.503761	test-:0.461261
[800]	train-:0.503338	test-:0.46147
[900]	train-:0.503233	test-:0.462196


In [78]:
pred = bst.predict(dtest)

In [79]:
ndcg(testY, pred, 5)

array([-0.00088118])

## linear model

In [217]:
from sklearn.linear_model import LinearRegression

In [218]:
lr = LinearRegression()
lr.fit(trainX, trainY)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [219]:
pred = lr.predict(testX)

In [220]:
from sklearn.metrics import mean_squared_error

In [221]:
mean_squared_error(pred, testY)

2.3206557810416113

In [222]:
ndcg(testY, pred,5)

array([ 0.04020345])

## preparing results

In [40]:
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
def get_text_from_query_by(i):
    text = queries[queries['QueryId']==i].values[0][1]
    return clean_query(text)

In [55]:
def to_nparray(res):
    print(res)
    result = np.zeros((len(res), 2))
    for i,row in enumerate(res):
        result[i,0] = row[1]
        result[i,1] = row[2]
    print(result)
    return result

In [43]:
def rank_block_by_dist(q_ids, d_ids):
    sim = []
    for q_id, d_id in zip(q_ids, d_ids):
        dv = get_sen_vector(docs[str(d_id)].text)
        qv = get_sen_vector(get_text_from_query_by(q_id))
        sim.append(cosine_similarity([dv],[qv]))
        
    return to_nparray(sorted(zip(sim, q_ids, d_ids), key=lambda key: key[0]))

In [61]:
def rank_block_by_pred(q_ids, d_ids):
    result = []
    for q_id, d_id in zip(q_ids, d_ids):
        d_vec = get_sen_vector(docs[str(d_id)].text).reshape(SIZE, 1)
        q_vec = get_sen_vector(get_text_from_query_by(q_id)).reshape(SIZE,1)
        r = np.hstack((q_vec, d_vec)).reshape(2 * SIZE)
        m = xgb.DMatrix(r)
        pred = bst.predict(m)
        result.append(pred)
        
    print(result)
        
    return to_nparray(sorted(zip(result, q_ids, d_ids), key=lambda key: key[0]))

In [62]:
def rank_tfidf(q_ids, d_ids):
    result = []
    for q_id, d_id in zip(q_ids, d_ids):
        q_v = np.mean(tv_queries.transform(queries[queries['QueryId']==q_id]), axis=0)
        d_v = np.mean(tv_docs.transform(docs[str(d_id)].text), axis=0)
        r = np.hstack((q_v, d_v)).reshape(2 * SIZE)
        m = xgb.DMatrix(r)
        pred = bst.predict(m)
        result.append(pred)
            
    return to_nparray(sorted(zip(result, q_ids, d_ids), key=lambda key: key[0], reverse=True))

## example for ranking in case of query absense in train

In [81]:
print(len(queries[queries['QueryId'] == 127]))

0


In [77]:
127 in queries['QueryId']

True

In [81]:
prev_q = -1
prev_i = -1
cnt = 0

result = np.zeros((len(rel_test),2))

for i,row in rel_test.iterrows():
    q_id = str(row['QueryId'])
    d_id = str(row['DocumentId'])
    
    if prev_q == -1:
        prev_q = q_id
        prev_i = i
        
    if prev_q != q_id:
        if len(queries[queries['QueryId'] == int(prev_q)]) > 0:
            print('ranking by pred')
            r = rank_tfidf(rel_test.iloc[prev_i:i,0], rel_test.iloc[prev_i:i,1])
            for row in r:
                result[cnt,0] = row[0]
                result[cnt,1] = row[1]
                cnt += 1
        else:
            print('simple rewirte')
            for j in range(prev_i, i):
                result[j,0] = rel_test.iloc[j,0]
                result[j,1] = rel_test.iloc[j,1]
                cnt += 1
                
        prev_q = q_id
        prev_i = i
    
    if i == len(rel_test)-1:
        i+= 1
        if len(queries[queries['QueryId'] == int(q_id)]) > 0:
            print('ranking by pred')
            r = rank_tfidf(rel_test.iloc[prev_i:i,0], rel_test.iloc[prev_i:i,1])
            for row in r:
                result[cnt,0] = row[0]
                result[cnt,1] = row[1]
                cnt += 1
        else:
            print('simple rewirte')
            for j in range(prev_i, i):
                result[j,0] = rel_test.iloc[j,0]
                result[j,1] = rel_test.iloc[j,1]
                cnt += 1

ranking by pred
[(array([ 5.0929451], dtype=float32), 126, 974), (array([ 2.59863329], dtype=float32), 126, 187), (array([ 1.73879623], dtype=float32), 126, 973), (array([ 1.53388178], dtype=float32), 126, 942), (array([ 0.69599879], dtype=float32), 126, 1326), (array([ 0.4590441], dtype=float32), 126, 972), (array([-0.16470325], dtype=float32), 126, 969), (array([-0.49719286], dtype=float32), 126, 970), (array([-1.39386308], dtype=float32), 126, 971)]
[[  126.   974.]
 [  126.   187.]
 [  126.   973.]
 [  126.   942.]
 [  126.  1326.]
 [  126.   972.]
 [  126.   969.]
 [  126.   970.]
 [  126.   971.]]
simple rewirte
ranking by pred
[(array([ 1.47048688], dtype=float32), 128, 990), (array([ 0.18342799], dtype=float32), 128, 985), (array([-1.00137365], dtype=float32), 128, 945)]
[[ 128.  990.]
 [ 128.  985.]
 [ 128.  945.]]
simple rewirte
ranking by pred
[(array([ 1.45838702], dtype=float32), 130, 859), (array([ 0.32401633], dtype=float32), 130, 1008), (array([ 0.04663098], dtype=float

[(array([ 3.16388679], dtype=float32), 152, 655), (array([ 2.89652061], dtype=float32), 152, 1062), (array([ 0.89000368], dtype=float32), 152, 569), (array([ 0.1872029], dtype=float32), 152, 572), (array([ 0.14488879], dtype=float32), 152, 687), (array([-0.29378313], dtype=float32), 152, 1077), (array([-1.14427674], dtype=float32), 152, 1076)]
[[  152.   655.]
 [  152.  1062.]
 [  152.   569.]
 [  152.   572.]
 [  152.   687.]
 [  152.  1077.]
 [  152.  1076.]]
ranking by pred
[(array([ 2.91569901], dtype=float32), 153, 1085), (array([ 1.88868499], dtype=float32), 153, 1083), (array([ 1.57192075], dtype=float32), 153, 1078), (array([ 1.04457748], dtype=float32), 153, 1081), (array([ 0.58489966], dtype=float32), 153, 1080), (array([-0.18531835], dtype=float32), 153, 1082), (array([-0.41898793], dtype=float32), 153, 1063), (array([-1.60840559], dtype=float32), 153, 1084)]
[[  153.  1085.]
 [  153.  1083.]
 [  153.  1078.]
 [  153.  1081.]
 [  153.  1080.]
 [  153.  1082.]
 [  153.  1063.

ValueError: Found array with 0 sample(s) (shape=(0, 60)) while a minimum of 1 is required by the normalize function.

In [194]:
tq = rel_test.iloc[0:9,0].values
td = rel_test.iloc[0:9,1].values
r2 = rank_block_by_pred(tq,td)

In [52]:
pd.unique(rel_test.iloc[0:len(rel_test), 0])

array([126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138,
       139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151,
       152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164,
       165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177,
       178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190,
       191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203,
       204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216,
       217, 218, 219, 220, 221, 222, 223, 224, 225])

In [55]:
pd.unique(queries.iloc[0:len(queries), 0])

array([  1,   2,   4,   8,   9,  10,  12,  13,  15,  18,  22,  23,  26,
        27,  29,  31,  32,  33,  34,  35,  39,  40,  41,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  61,  62,  66,  67,  68,
        69,  71,  72,  74,  79,  80,  81,  82,  83,  84,  85,  86,  87,
        93,  94,  95,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106,
       107, 108, 109, 110, 111, 112, 113, 114, 116, 118, 119, 120, 121,
       122, 123, 126, 128, 130, 131, 132, 133, 135, 136, 137, 138, 139,
       140, 141, 142, 143, 145, 146, 147, 148, 149, 150, 152, 153, 154,
       155, 156, 157, 158, 160, 161, 163, 164, 165, 167, 168, 169, 170,
       171, 173, 175, 176, 177, 181, 182, 183, 184, 187, 189, 190, 196,
       200, 201, 202, 203, 204, 205, 206, 208, 209, 210, 211, 212, 213,
       214, 215, 216, 217, 218, 219, 223, 224, 225, 226, 227, 230, 231,
       232, 233, 234, 241, 245, 246, 247, 250, 251, 252, 253, 254, 255,
       257, 259, 261, 264, 265, 266, 267, 268, 269, 272, 273, 27

In [471]:
result.shape

(847, 2)

In [57]:
out = pd.DataFrame(result, dtype=np.uint8)

In [58]:
out.to_csv(path_or_buf='pred.csv', columns=(0,1), header=('QueryId','DocumentId'), index=False)